<h4 align = center> Dr. Jesús Martí Gavilá. Departamento de Ingeniería Cartográfica, Geodesia y fotogrametría </h4>
<font color="blue" ><h1 align=center>PRÁCTICA 8</h1> <p> <h1 align=center> CURSO BÁSICO PYTHON: FOLIUM</h1></font>

<a href="http://www.upv.es/titulaciones/GTI/indexc.html"><img src = "http://www.upv.es/imagenes/marcaUPVN1.png" width =200 ></a>

<h2 align=center>Grado Tecnologías Interactivas | Tecnologías de la Información Geográfica</h2>



<div class="alert alert-block alert-info" style="margin-top: 20px">
<b>FOLIUM</b> es una biblioteca de visualización de Python que se desarrolló con el único fin de visualizar datos geoespaciales. Es una biblioteca completamente gratuita. <p>  
<h2>Tabla de Contenido</h2>   
    <ul>
        <li>
            <a>Acerca de los conjuntos de datos (DataFrame)</a>
        </li>
        <li>
            <a>Mapa Puntual</a>
        </li>
        <li>
            <a>Mapa de calor</a>
        </li>
        <li>
            <a>Mapa de calor con función tiempo</a>
        </li>
        <li>
            <a>Tareas a realizar</a>
        </li>
    </ul>
<p>
<b> Base de datos: </b> <a href="https://datos.madrid.es/portal/site/egob"> https://datos.madrid.es/portal/site/egob </a>
<p>
<b> Documentación: </b> <a href="https://python-visualization.github.io/folium/"> https://python-visualization.github.io/folium/</a><p>    
<p<<p>





<H3> Alumnos: </H3> Sento Marcos Ibarra

<h1 align=center><font color="green" >Estudio de la accidentalidad de la ciudad de Madrid mediante mapa de calor y puntual.</font></h1><p>
    
<h2>Caso de estudio</h2>
Crear un mapa de calor y puntual de los accidentes originados por vehiculos y bicicletas en el año 2022

<h2><font color="blue" >CREACION BASE DE DATOS ESTUDIO</font></h2><p>

In [ ]:
# Librerias necesarias
import openpyxl
import pandas as pd
import numpy as np
import folium
from folium import plugins
import webbrowser
from pyproj import Transformer

In [ ]:
# Creamos la Base de datos
base_datos= pd.read_excel('2022_Accidentalidad.xlsx')
base_datos.head(5)

# Preparación Base de datos

In [ ]:
print(base_datos.dtypes) # imprime el tipo de campo
print("Numero de registros =" , len( base_datos)) # imprime el tamaño de la base de datos

In [ ]:
# Elimina los registros en blanco en los campos de coordenadas
accidentes_df = base_datos.dropna(subset=['coordenada_x_utm', 'coordenada_y_utm'])

# Transformación de coordenadas

La geolocalización en el fichero de accidentes del ayuntamiento de Madrid se realiza con coordenadas UTM ETRS89 en el Huso 30. Para representar coordenadas en FOLIUM se necesitan coordenadas geográficas ETRS89

In [ ]:
# Establecer los sistemas de transformación epsg:xxxx de entrada y epsg:yyyyy de salida
transformacion = Transformer.from_crs('epsg:25830','epsg:4258',always_xy=True) 
puntos = list(zip(accidentes_df.coordenada_x_utm,accidentes_df.coordenada_y_utm)) # Crea una lista con todos los puntos

# Se recomienda trabajar con una copia independiente
accidentes_df = accidentes_df.copy()
coorgeo = np.array(list(transformacion.itransform(puntos)))
accidentes_df.loc[:,'longitud'] = coorgeo[:,0]
accidentes_df.loc[:,'latitud'] = coorgeo[:,1]

accidentes_df.head()

# Selección de registros base de datos

In [ ]:
print(accidentes_df["tipo_persona"].unique())

In [ ]:
# Crear una nueva Dataframe que cumple las condiciones establecidas
acci_conductor = accidentes_df.loc[(accidentes_df['tipo_persona']=='Conductor')] 
                          
# si quisieramos introduccir más condicionantes --- & (accidentes_df['tipo_vehiculo']=='Turismo')& (accidentes_df['rango_edad']=='De 18 a 20 años')]

<h2><font color="blue" >Mapa Puntual (MarkerCluster)</font></h2><p>

Los mapas puntuales permiten plasmar variables georeferenciadas, para un mejor entendimiento por parte del usuario de la distribucción de los mismas.<p>
    
<b> Diferentes plugins de Folium: </b> <a href="https://python-visualization.github.io/folium/plugins.html"> https://python-visualization.github.io/folium/plugins.html </a>    
<p>

<b> Iconos con prefijo 'fa': </b> <a href="https://fontawesome.com/v5.15/icons?d=gallery&p=2&m=free"> https://fontawesome.com/v5.15/icons?d=gallery&p=2&m=free</a>    

In [ ]:
print(accidentes_df["tipo_vehiculo"].unique())

In [ ]:
# Crear el mapa base donde representar
mapa_accidentes = folium.Map(location=(40.43,-3.65), tiles = 'OpenStreetMap', zoom_start = 12)

# Crear diferentes dataframes, con tipo _persona= Conductor y Vehiculos= Turismo y Bicicleta
coche_df=acci_conductor.loc[(acci_conductor['tipo_vehiculo']=='Turismo')] 
bici_df=acci_conductor.loc[(acci_conductor['tipo_vehiculo']=='Bicicleta')]

# crear un objeto de grupo de marcas para los incidentes en el DataFrame
coches = plugins.MarkerCluster( name="Accidentes_coche",).add_to(mapa_accidentes)
bicis = plugins.MarkerCluster( name="Accidentes_bicis",).add_to(mapa_accidentes)

# procesar el DataFrame y agregar cada punto de datos al grupo de marcas creado anteriormente
for lat, lng, label, in zip(coche_df.latitud, coche_df.longitud, coche_df['num_expediente']): #dos formas diferentes de llamar a un campo
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color="orange", icon="car", prefix = 'fa'),
        popup=label,
        ).add_to(coches)

for lat, lng, label, in zip(bici_df.latitud, bici_df.longitud, bici_df['num_expediente']): #dos formas diferentes de llamar a un campo
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color="blue", icon="bicycle", prefix = 'fa'),
        popup=label,
        ).add_to(bicis)

#Añadir controles
folium.LayerControl().add_to(mapa_accidentes)
draw = plugins.Draw(export=True)
draw.add_to(mapa_accidentes)

#Muestra mapa
#mapa_accidentes

# Salvar WebMapping
mapa_accidentes.save("madrid_accidentes.html")
webbrowser.open_new_tab('madrid_accidentes.html')

 <h1 align=center><font color="green" >Ubicación de contenedores vidrio y envases en Madrid. Mapa puntual.</font></h1><p> 

In [ ]:
'''
La base de datos es Contenedores_varios.csv, utilizamos ahora un fichero CSV
Es opcional colocar el tipo de separador y la codificación. Por defecto es , y UTF-8 peró hay ficheros csv
con formato sep=';', encoding = 'cp1252')
'''
contenedores_df = pd.read_csv('Contenedores_varios.csv', sep=';',low_memory = False)
contenedores_df.head()

In [ ]:
print(contenedores_df["Tipo Contenedor"].unique())
print(contenedores_df.dtypes) # imprime el tipo de campo

In [ ]:
# Crear el mapa base donde representar
mapa_contenedores = folium.Map(location=(40.43,-3.65), tiles = 'OpenStreetMap', zoom_start = 12)

# Crear diferentes dataframes 
vidrio_df=contenedores_df.loc[(contenedores_df['Tipo Contenedor']=='VIDRIO')] 
envases_df=contenedores_df.loc[(contenedores_df['Tipo Contenedor']=='ENVASES')] 

# crear un objeto de grupo de marcas para los incidentes en el DataFrame
vidrio = plugins.MarkerCluster( name="Contenedores Vidrio").add_to(mapa_contenedores)
envases = plugins.MarkerCluster( name = 'Contenedores envases').add_to(mapa_contenedores)

# procesar el DataFrame y agregar cada punto de datos al grupo de marcas creado anteriormente
for lat, lng, label in zip(vidrio_df.LATITUD, vidrio_df.LONGITUD, vidrio_df['Descripcion Modelo']):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color="green", icon="info-sign"),
        popup=label,
        ).add_to(vidrio)

for lat, lng, label in zip(envases_df.LATITUD, envases_df.LONGITUD, envases_df['Descripcion Modelo']):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color="orange", icon="info-sign"),
        popup=label,
        ).add_to(envases)

# Añadir control de mapas
folium.LayerControl().add_to(mapa_contenedores)

#Muestra mapa
mapa_contenedores

# Crear mapas
# mapa_contenedores.save("madrid_contenedores.html")
# webbrowser.open_new_tab('madrid_contenedores.html')

<h2><font color="blue" >Mapa de Calor (HeatMap)</font></h2><p>

Los mapas de calor, o heatmaps, son un tipo de representación dentro de de la simbología de mapas que nos permite mostrar al usuario los puntos calientes o conjuntos de datos más relevantes dentro de una nube de puntos. Con una combinación de opciones de simbología y datos cuantitativos provenientes de base de datos georeferenciadas, obtendras mapas que reflejan concentraciones o distribuciones espaciales de variables no continuas en el espacio.

In [ ]:
# Utilizaremos el pluging HeatMap
# la lista de datos puede ser de 2 formas (Latitud, Longitud) o (Latitud, Longitud y Peso).
# Mapa de calor de accidentes de Madrid
mapa_calor = folium.Map(location=(40.43,-3.65), tiles = 'cartodb positron', zoom_start = 12)
plugins.HeatMap(data=acci_conductor[['latitud', 'longitud']], radius=15, use_local_extrema=True, name="Mapa de calor").add_to(mapa_calor)
folium.LayerControl().add_to(mapa_calor)
mapa_calor

In [ ]:
# publicar el mapa
mapa_calor.save("heatmap_acci.html")
webbrowser.open_new_tab('heatmap_acci.html') #Abre el HTML en una ventana del navegador

 <h2><font color="blue" >Mapa de calor con evolución en el tiempo (HeatMapWithTime) </font></h2><p>
Este plugin nos permite generar mapas de calor de diferentes periodos de tiempo especificados y representarlos mediante automatizacion con un play que nos permite represesentar la evolución del fenómeno

In [ ]:
# Crear una nueva Dataframe que cumpla unas determinadas condiciones
acci_df=accidentes_df.loc[(accidentes_df['tipo_persona']=='Conductor') & (accidentes_df['tipo_vehiculo']=='Turismo')]

In [ ]:
# Crear un campo en el Dataframe con el valor del mes extraido de la fecha
import datetime
mes = pd.DatetimeIndex(acci_df['fecha']).month 
acci_df = acci_df.assign(mes = mes)  
acci_df.head()

In [ ]:
# Generar lista para el slider del visualizador
lista_tiempo = ['Enero','Febrero','Marzo','Abril','Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre','Diciembre']

In [ ]:
lista_peso = [] # Crear una lista vacia
acci_df = acci_df.assign(peso = 1)  # crear un campo nuevo con valor por defecto 1

# Rellenar la lista con los campos (Latitud, Longitud y Peso) necesarios para HeatMap
for x in acci_df['mes'].sort_values().unique():
   lista_peso.append(acci_df.loc[acci_df['mes']==x,   # Agrupar los valores con latitud y longitud igual 
                  ['latitud','longitud','peso']].groupby(['latitud','longitud']).sum().reset_index().values.tolist())

In [ ]:
print(lista_peso     )

In [ ]:
# Generar el mapa
madrid_map = folium.Map(location=(40.43,-3.65), control_scale = True, tiles = "cartodb positron", zoom_start = 12)

# Ejecutar el pluging HeatMapWithTime
plugins.HeatMapWithTime(lista_peso, radius = 30, index = lista_tiempo,
                        auto_play = False, min_opacity = 0.5, max_opacity = 1, use_local_extrema=True, name="Accidentes_Turismo_2022").add_to(madrid_map)

# publica el mapa
folium.LayerControl().add_to(madrid_map)
madrid_map.save("madrid_acci.html")
webbrowser.open_new_tab('madrid_acci.html')

<h2><font color="blue" >Mapa Dual (DualMap) </font></h2><p>
 Este plugin nos permite en una misma ventana Html tener 2 mapas
 <p>

 <h1 align=center><font color="green" >Ubicación de contenedores vidrio y envases en Madrid. Mapa dual puntual.</font></h1><p>   

In [ ]:
'''
La base de datos es Contenedores_varios.csv, utilizamos ahora un fichero CSV
Es opcional colocar el tipo de separador y la codificación. Por defecto es , y UTF-8 peró hay ficheros csv
con formato sep=';', encoding = 'cp1252')
'''
contenedores_df = pd.read_csv('Contenedores_varios.csv', sep=';',low_memory = False)
contenedores_df.head()

In [ ]:
print(contenedores_df["Tipo Contenedor"].unique())

In [ ]:
# Crear el mapa dual dondce representar los elementos
mapa_dual = plugins.DualMap(location=[40.4, -3.65], tiles=None, zoom_start=20)

# map tiles
folium.TileLayer('OpenStreetMap').add_to(mapa_dual.m1)
folium.TileLayer('CartoDB Positron').add_to(mapa_dual.m2)

# Crear diferentes dataframes, con Tipo Contenedor= VIDRIO y ENVASES
vidrio_df=contenedores_df.loc[(contenedores_df['Tipo Contenedor']=='VIDRIO')] 
envases_df=contenedores_df.loc[(contenedores_df['Tipo Contenedor']=='ENVASES')] 

# crear un objeto de grupo de marcas para los diferentes contenedores en el DataFrame
vidrio = plugins.MarkerCluster( name="Contenedores Vidrio").add_to(mapa_dual.m1)
envases = plugins.MarkerCluster( name = 'Contenedores envases').add_to(mapa_dual.m2)

for lat, lng, label in zip(vidrio_df.LATITUD, vidrio_df.LONGITUD, vidrio_df['Descripcion Modelo']):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color="green", icon="info-sign"),
        popup=label,
        ).add_to(vidrio)

for lat, lng, label in zip(envases_df.LATITUD, envases_df.LONGITUD, envases_df['Descripcion Modelo']):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color="orange", icon="info-sign"),
        popup=label,
        ).add_to(envases)

# Añadir control de mapas
folium.LayerControl().add_to(mapa_dual)

# Crear mapas
mapa_dual.save("madrid_dual.html")
webbrowser.open_new_tab('madrid_dual.html')

<h2><font color="red" >TAREA 1</font></h2><p>
<h3> Realizar un webMap de la gestión de residuos en la ciudad de Madrid.</h3> <p>
Crear capas diferenciadas: Mapa de calor de los residuos vidrio, Mapa de calor de los residuos envases y la ubicación de los contenedores diferenciados por el tipo de contenedor (vidrio, envases, papel, organico).<P>
    <h4>Guardar como: contenedores_"apellido alumno".html</h4>

In [2]:
# Librerias necesarias
import openpyxl
import pandas as pd
import numpy as np
import folium
from folium import plugins
import webbrowser
from pyproj import Transformer

In [3]:
#solo un webmap con lo solicitado por capas
# Abrir la base de datos Contenedores_varios.csv
contenedores_df = pd.read_csv('Contenedores_varios.csv', sep=';',low_memory = False)
contenedores_df.head()

,Código Interno del Situad,Tipo Contenedor,Modelo,Descripcion Modelo,Cantidad,Lote,Distrito,Barrio,Tipo Vía,Nombre,Número,COORDENADA X,COORDENADA Y,LONGITUD,LATITUD,DIRECCION
0,168755,ENVASES,NaN,Env 3200 CL,1.0,1,CENTRO,NaN,PLAZA,SALESAS,9.0,441062.99,4475052.45,-3.694768,40.424017,PLAZA SALESAS. 9
1,168755,VIDRIO,NaN,V 2200 CL VACRI,1.0,1,CENTRO,NaN,PLAZA,SALESAS,9.0,441062.99,4475052.45,-3.694768,40.424017,PLAZA SALESAS. 9
2,168753,VIDRIO,NaN,V 2200 CL,1.0,1,CENTRO,NaN,CALLE,MEJIA LEQUERICA,20.0,440680.69,4475502.55,-3.699317,40.428045,CALLE MEJIA LEQUERICA. 20
3,170313,RESTO,NaN,Res 800,1.0,1,FUENCARRAL-EL PARDO,NaN,CALLE,ISLA DE JAVA,55.0,441852.02,4482612.15,-3.686160,40.492174,CALLE ISLA DE JAVA. 55
4,169874,ENVASES,NaN,Env 800,1.0,1,FUENCARRAL-EL PARDO,NaN,AVENIDA,CARDENAL HERRERA ORIA,271.0,438643.32,4481240.57,-3.723888,40.479587,AVENIDA CARDENAL HERRERA ORIA. 271


In [4]:
# Crear diferentes dataframes para los residuos indicados
vidrio_df=contenedores_df.loc[(contenedores_df['Tipo Contenedor']=='VIDRIO')] 
envases_df=contenedores_df.loc[(contenedores_df['Tipo Contenedor']=='ENVASES')]
organico_df=contenedores_df.loc[(contenedores_df['Tipo Contenedor']=='ORGANICA')] 
papel_df=contenedores_df.loc[(contenedores_df['Tipo Contenedor']=='PAPEL-CARTON')] 


In [ ]:
# Generar mapa
mapa_contenedor = folium.Map(location=(40.43,-3.65), tiles = 'OpenStreetMap', zoom_start = 12)
# Generar mapas de calor
plugins.HeatMap(data=vidrio_df[['LATITUD', 'LONGITUD']], radius=15, use_local_extrema=True, name="Mapa de calor vidrio").add_to(mapa_contenedor)
plugins.HeatMap(data=envases_df[['LATITUD', 'LONGITUD']], radius=15, use_local_extrema=True, name="Mapa de calor envases").add_to(mapa_contenedor)

# crear objetos de grupo de marcas para los incidentes en el DataFrame
papel = plugins.MarkerCluster( name="Contenedores Papel-Carton").add_to(mapa_contenedor)
organico = plugins.MarkerCluster( name = 'Contenedores organico').add_to(mapa_contenedor)
vidrio = plugins.MarkerCluster( name="Contenedores vidrio").add_to(mapa_contenedor)
envases = plugins.MarkerCluster( name = 'Contenedores envases').add_to(mapa_contenedor)
# procesar el DataFrame y agregue cada punto de datos al grupo de marca
for lat, lng, label in zip(papel_df.LATITUD, papel_df.LONGITUD, papel_df['Descripcion Modelo']):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color="green", icon="info-sign"),
        popup=label,
        ).add_to(papel)
    
for lat, lng, label in zip(organico_df.LATITUD, organico_df.LONGITUD, organico_df['Descripcion Modelo']):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color="orange", icon="info-sign"),
        popup=label,
        ).add_to(organico) 
for lat, lng, label in zip(vidrio_df.LATITUD, vidrio_df.LONGITUD, vidrio_df['Descripcion Modelo']):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color="green", icon="info-sign"),
        popup=label,
        ).add_to(vidrio)
for lat, lng, label in zip(envases_df.LATITUD, envases_df.LONGITUD, envases_df['Descripcion Modelo']):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color="green", icon="info-sign"),
        popup=label,
        ).add_to(envases)  
folium.LayerControl().add_to(mapa_contenedor)
# publicar el mapa. Contenedores_"apellido alumno".html
mapa_contenedor.save("madrid_contenedores_Marcos_Ibarra_Sento.html")
webbrowser.open_new_tab('madrid_contenedores_Marcos_Ibarra_Sento.html')

### <h2><font color="red" >TAREA 2</font></h2><p>
<h3> Realizar webmapping con mapas de calor de accidentes en 2023 distribuidos por meses (HeatMapWithTime)</h3><p>
Realizar varios mapas de calor de accidentes producidos en la ciudad de Madrid, a lo largo del 2023 y distribuido por meses, que se han visto involucrados conductores diferenciados por sexo (Elegir: Hombre o Mujer) y con vehiculos distintos al turismo.

In [6]:
# Cargar y limpiar tabla de blancos
base_datos= pd.read_excel('2023_Accidentalidad.xlsx')
accidentes_df = base_datos.dropna(subset=['coordenada_x_utm', 'coordenada_y_utm'])

In [7]:
# Transformar coordenadas
transformacion = Transformer.from_crs('epsg:25830','epsg:4258',always_xy=True) 
puntos = list(zip(accidentes_df.coordenada_x_utm,accidentes_df.coordenada_y_utm)) # Crea una lista con todos los puntos
# Se recomienda trabajar con una copia independiente
accidentes_df = accidentes_df.copy()
coorgeo = np.array(list(transformacion.itransform(puntos)))
accidentes_df.loc[:,'longitud'] = coorgeo[:,0]
accidentes_df.loc[:,'latitud'] = coorgeo[:,1]

In [8]:
# Crear una nueva Dataframe que cumpla unas determinadas condiciones
acci_conductor = accidentes_df.loc[(accidentes_df['sexo']=='Hombre')&(accidentes_df['tipo_vehiculo']!='Turismo')]

In [9]:
# Crear un campo en el Dataframe con el valor del mes extraido de la fecha
import datetime
mes = pd.DatetimeIndex(accidentes_df['fecha']).month 
accidentes_df = accidentes_df.assign(mes = mes)  
accidentes_df.head()

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,...,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,longitud,latitud,mes
0,2023S000001,2023-01-01,01:15:00,"AVDA. ALFONSO XIII, 33",33,5.0,CHAMARTÍN,Alcance,Despejado,Todo terreno,...,Mujer,14.0,Sin asistencia sanitaria,443397.166,4478129.388,N,NaN,-3.667528,40.451898,1
1,2023S000001,2023-01-01,01:15:00,"AVDA. ALFONSO XIII, 33",33,5.0,CHAMARTÍN,Alcance,Despejado,Todo terreno,...,Hombre,14.0,Sin asistencia sanitaria,443397.166,4478129.388,N,NaN,-3.667528,40.451898,1
2,2023S000001,2023-01-01,01:15:00,"AVDA. ALFONSO XIII, 33",33,5.0,CHAMARTÍN,Alcance,Despejado,Todo terreno,...,Hombre,14.0,Sin asistencia sanitaria,443397.166,4478129.388,N,NaN,-3.667528,40.451898,1
3,2023S000001,2023-01-01,01:15:00,"AVDA. ALFONSO XIII, 33",33,5.0,CHAMARTÍN,Alcance,Despejado,Todo terreno,...,Mujer,7.0,Asistencia sanitaria sólo en el lugar del acci...,443397.166,4478129.388,N,NaN,-3.667528,40.451898,1
4,2023S000001,2023-01-01,01:15:00,"AVDA. ALFONSO XIII, 33",33,5.0,CHAMARTÍN,Alcance,Despejado,Todo terreno,...,Hombre,7.0,Asistencia sanitaria sólo en el lugar del acci...,443397.166,4478129.388,N,NaN,-3.667528,40.451898,1


In [10]:
# lista para el slider del visualizador
lista_tiempo = ['Enero','Febrero','Marzo','Abril','Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre','Diciembre']

In [11]:
# Crear una lista vacia
lista_peso = [] # Crear una lista vacia
# crear un campo nuevo con valor por defecto 1
accidentes_df = accidentes_df.assign(peso = 1)  # crear un campo nuevo con valor por defecto 1
# Rellenar la lista con los campos (Latitud, Longitud y Peso) necesarios para HeatMap
for x in accidentes_df['mes'].sort_values().unique():
   lista_peso.append(accidentes_df.loc[accidentes_df['mes']==x,   # Agrupar los valores con latitud y longitud igual 
                  ['latitud','longitud','peso']].groupby(['latitud','longitud']).sum().reset_index().values.tolist())

In [ ]:
# Generar el mapa
madrid_map = folium.Map(location=(40.43,-3.65), control_scale = True, tiles = "cartodb positron", zoom_start = 12)
# Ejecutar el pluging HeatMapWithTime
plugins.HeatMapWithTime(lista_peso, radius = 30, index = lista_tiempo,
                        auto_play = False, min_opacity = 0.5, max_opacity = 1, use_local_extrema=True, name="Accidentes_Turismo_Hombres_2023").add_to(madrid_map)
# publica el mapa. Accidentes_Madrid_"apellido alumno".html
folium.LayerControl().add_to(madrid_map)
madrid_map.save("madrid_acci_Marcos_Ibarra_Sento.html")
webbrowser.open_new_tab('madrid_acci_Marcos_Ibarra_Sento.html')